---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [134]:
import pandas as pd
import re
import datetime
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

In [135]:
def date_sorter():
    
    # Your code here
    
    a1 = df.str.extractall(r'(\d{1,2})[-/](\d{1,2})[-/](\d{2})\b')
    a2 = df.str.extractall(r'(\d{1,2})[-/](\d{1,2})[-/](\d{4})\b')
    a = pd.concat([a1,a2])
    a.reset_index(inplace = True)
    a_index = a['level_0']
    
    b = df.str.extractall(r'[^/0-9](\d{1,2})[/](\d{4})')
    b.reset_index(inplace = True)
    b_index = b['level_0']
    
    c = df.str.extractall(r'^(\d{1,2})[-/](\d{4})')
    c.reset_index(inplace = True)
    c_index = c['level_0']
    
    d = df.str.extractall(r'(\d{1,2} )?(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z,.]* (\d{4})')
    d.reset_index(inplace = True)
    d_index = d['level_0']
    
    e = df.str.extractall(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z,.]* (\d{1,2})[,]* (\d{4})')
    e.reset_index(inplace = True)
    e_index = e['level_0']
    
    f = df.str.extractall(r'^(\d{4})\D')
    g = df.str.extractall(r'\D(\d{4})\D')
    h = pd.concat([g,f])
    h.reset_index(inplace = True)
    h_index = h['level_0']
    temp = []
    for i in h_index:
        if not((i in a_index.values) | (i in b_index.values) | (i in c_index.values) | (i in d_index.values) | (i in e_index.values)):
            temp.append(i)
    h = h[h['level_0'].isin(temp)]
    h_index = h['level_0']
    
    a.columns = ['serial_no.', 'match', 'month', 'day', 'year']
    a['year'] = a['year'].apply(str)
    a['year'] = a['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    
    b.columns = ['serial_no.', 'match', 'month', 'year']
    b['day'] = 1
    
    c.columns = ['serial_no.', 'match', 'month', 'year']
    c['day'] = 1
    
    h.columns = ['serial_no.', 'match', 'year']
    h['month'] = 1
    h['day'] = 1
    
    months = {
        'Jan' : 1,
        'Feb' : 2,
        'Mar' : 3,
        'Apr' : 4,
        'May' : 5,
        'Jun' : 6,
        'Jul' : 7,
        'Aug' : 8,
        'Sep' : 9,
        'Oct' : 10,
        'Nov' : 11,
        'Dec' : 12
    }
    
    e.columns = ['serial_no.', 'match', 'month', 'day', 'year']
    e['month'] = e['month'].apply(lambda x: months[x])
    
    d.columns = ['serial_no.', 'match', 'day', 'month', 'year']
    d['month'] = d['month'].apply(lambda x: months[x])
    d['day'] = d['day'].fillna(1)
    
    dates = pd.concat([a,b,c,d,e,h])
    dates['date'] = pd.to_datetime(dates['month'].apply(str) + '/' + dates['day'].apply(str) + '/' + dates['year'].apply(str))
    
    result = dates.sort_values(by = 'date').reset_index().drop('index', axis = 1)['serial_no.']
    
    return result
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     415
17     335
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    480
478    286
479    431
480    279
481    198
482    463
483    381
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: serial_no., Length: 500, dtype: int64